## Zadania do wykonania

### Zadanie 1

Napisz program typu `CRUD` zbierający sygnały (zdarzenia) z różnych źródeł. Tabela powinna posiadać:

- dokładną datę zdarzenia,
- źródło zdarzenia,
- typ zdarzenia (informacja, ostrzeżenie, błąd, błąd krytyczny),
- dodatkowe dane,
- identyfikator (adres może być np. IP).

Na kolumnę typ zdarzenia powinien zostać założony indeks, a wartości powinny być ograniczone do tych wyliczonych w nawiasie.

### Zadanie 2

Do zadania 2 z laboratorium 1 dodaj obsługę bazy danych.

### Zadanie 3

Napisz klasę do logowania błędów. Zdarzenia powinny być logowane do bazy danych jako *bulk load*. Klasa powinna obsługiwać dwie bazy danych - główną (dowolna relacyjna baza danych) i lokalna (na dane tymczasowe). Schemat logowania jest następujący:
- jeśli baza danych (operacyjna) jest niedostępna logowanie powinno nastąpić do lokalnej bazy danych,
- jeśli połączenie do głównej bazy głównej zostanie przywrócone, klasa powinna przenieść wszystkie dane z lokalnej bazy danych do głównej.

Należy pamiętać o unikalności klucza głównego.

In [14]:
//Zadanie 1 Wykonali: Bartosz, Patryk S

using FluentNHibernate.Cfg;
using FluentNHibernate.Cfg.Db;
using NHibernate;
using NHibernate.Tool.hbm2ddl;
using FluentNHibernate.Mapping;
using System.Data.SQLite;
using NHibernate.Cfg;

public class Event
{
    public virtual int Id { get; set; }
    public virtual DateTime Date { get; set; }
    public virtual string? Source { get; set; }
    public virtual EventTypes EventType { get; set; }
    public virtual string? AdditionalData { get; set; }
    public virtual string? Identifier { get; set; }

    public override string ToString()
        {
            return "\nId: "+Id+"\nData: "+Date + "\nŹródło: "+Source+"\nTyp: "+EventType+"\nDodatkowe informacje: "+AdditionalData+"\nIdentyfikator: "+Identifier;
        }

    public enum EventTypes
    {
        Informacja,
        Ostrzeżenie,
        Błąd,
        BłądKrytyczny
    }
}

public class EventMap : ClassMap<Event>
{
    public EventMap()
    {
        Id(x => x.Id);
        Map(x => x.Date);
        Map(x => x.Source);
        Map(x => x.EventType);
        Map(x => x.AdditionalData);
        Map(x => x.Identifier);
        Table("Events");
    }
}

static void BuildSchema(Configuration config)
{
    new SchemaExport(config)
        .Create(false, true);
}

static ISessionFactory CreateSessionFactory(params Type[] mappingTypes)
{
    return Fluently.Configure()
        .Database(SQLiteConfiguration.Standard
            .UsingFile("Events.sqlite"))
        .Mappings(m => {
            foreach(var mappingType in mappingTypes) 
                m.FluentMappings.Add(mappingType);
        })
        .ExposeConfiguration(BuildSchema)
        .BuildSessionFactory();
}

var sessionFactory = CreateSessionFactory(typeof(EventMap));

using (var session = sessionFactory.OpenSession()) {
        using (var transaction = session.BeginTransaction()) {  

            var eventSignal1  = new Event() {
                Date = DateTime.Now,
                Source = "System - Windows 11",
                EventType = (Event.EventTypes) 1,
                AdditionalData = "Wersja opraogramowania może nie być już wspierana",
                Identifier = "192.168.0.2"
            };
            session.Save(eventSignal1);

            var eventSignal2 = new Event {
                Date = DateTime.Now,
                Source = "System - Windows 7",
                EventType = (Event.EventTypes) 2,
                AdditionalData = "Nie znaleziono pliku",
                Identifier = null
            };
            session.SaveOrUpdate(eventSignal2);

            transaction.Commit();
        }

        session.Close();
}

using (var session = sessionFactory.OpenSession()) {
        using (var transaction = session.BeginTransaction()) {

            var eventSignal3 = new Event {
                Date = DateTime.Now,
                Source = "System - Windows 7",
                EventType = (Event.EventTypes) 2,
                AdditionalData = "Brak wymaganych uprawnień",
                Identifier = "192.168.0.5"
            };

            session.SaveOrUpdate(eventSignal3);
            
            var retrievedEvent = session.Get<Event>(3);
            retrievedEvent.Source = "System - Linux";
            session.Update(retrievedEvent);
            retrievedEvent = session.Get<Event>(2);
            session.Delete(retrievedEvent);

            transaction.Commit();

        }

    session.Query<Event>().ToList().ForEach(Console.WriteLine);
}




Id: 1
Data: 03.03.2023 17:04:19
Źródło: System - Windows 11
Typ: Ostrzeżenie
Dodatkowe informacje: Wersja opraogramowania może nie być już wspierana
Identyfikator: 192.168.0.2

Id: 3
Data: 03.03.2023 17:04:19
Źródło: System - Linux
Typ: Błąd
Dodatkowe informacje: Brak wymaganych uprawnień
Identyfikator: 192.168.0.5


In [1]:
// Zadanie 2 Wykonali: Bartosz, Patryk S

using System.Configuration;
using System;
using System.IO;
using NHibernate;
using FluentNHibernate.Mapping;
using NPOI.SS.Formula.Functions;

public class EmployeeMap : ClassMap<Employee<T>>
{
    public EmployeeMap()
    {
        Id(x => x.Id);
        Map(x => x.FirstName);
        Map(x => x.LastName);
        Map(x => x.Position);
        Map(x => x.Department);
        Map(x => x.PhoneNumber);
        Table("Employees");
    }
}

public class SessionFactoryX
{
    private static ISessionFactory _sessionFactory;

    private static ISessionFactory SessionFactory
    {
        get
        {
            if (_sessionFactory == null)
            {
                var configuration = new NHibernate.Cfg.Configuration();
                configuration.Configure();
                configuration.AddAssembly(typeof(EmployeeMap).Assembly);
                _sessionFactory = configuration.BuildSessionFactory();
            }
            return _sessionFactory;
        }
    }

    public static ISession OpenSession()
    {
        return SessionFactory.OpenSession();
    }
}



public class Employee<T>
{
    
    public T Id { get; set; }
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string Position { get; set; }
    public string Department { get; set; }
    public string PhoneNumber { get; set; }

    public Employee(T id, string firstName, string lastName, string position, string department, string phoneNumber)
    {
        Id = id;
        FirstName = firstName;
        LastName = lastName;
        Position = position;
        Department = department;
        PhoneNumber = phoneNumber;
    }

    public bool Validate()
    {
        bool isValid = true;

        if (string.IsNullOrEmpty(FirstName))
        {
            throw new ArgumentException("Imie nie może być puste.");
            isValid = false;
        }
        if (string.IsNullOrEmpty(LastName))
        {
            throw new ArgumentException("Nazwwisko nie może być puste.");
            isValid = false;
        }
        if (string.IsNullOrEmpty(Position))
        {
            throw new ArgumentException("Stanowisko nie może być puste.");
            isValid = false;
        }
        if (string.IsNullOrEmpty(Department))
        {
            throw new ArgumentException("Departament nie może być pusty.");
            isValid = false;
        }
        if (string.IsNullOrEmpty(PhoneNumber))
        {
            throw new ArgumentException("Numer telefonu nie może być pusty.");
            isValid = false;
        }

        return isValid;
    }

    public void Show()
    {
        Console.WriteLine(Format());
    }

    public bool IsMatch(Employee<T> employee)
    {
        return Id.Equals(employee.Id) &&
            FirstName == employee.FirstName &&
            LastName == employee.LastName &&
            Position == employee.Position &&
            Department == employee.Department &&
            PhoneNumber == employee.PhoneNumber;
    }

    public string Format()
    {
        return $"\nId: {Id}\nImię: {FirstName}\nNazwisko: {LastName}\nStanowisko: {Position}\nDepartament: {Department}\nNumer telefonu: {PhoneNumber}";
    }

}

public class EmployeeDirectory<T>
{
    private List<Employee<T>> _employees;
    public EmployeeDirectory()
    {
        _employees = new List<Employee<T>>();
    }

    public void Add(Employee<T> employee)
    {
        if (employee.Validate())
        {
            _employees.Add(employee);
        }
    }

    public void Remove(Employee<T> employee)
    {
        _employees.Remove(employee);
    }

    public void Show()
    {
        foreach (Employee<T> employee in _employees)
        {
            employee.Show();
        }
    }

    public Employee<T> Search(Employee<T> employee)
    {
        foreach (Employee<T> e in _employees)
        {
            if (e.IsMatch(employee))
            {
                return e;
            }
        }

        return null;
    }

    public void Save(Employee<T> employee)
    {
        using (ISession session = SessionFactoryX.OpenSession())
        {
            using (ITransaction transaction = session.BeginTransaction())
            {
                session.SaveOrUpdate(employee);
                transaction.Commit();
            }
        }
    }

    public List<Employee<T>> GetAll()
    {
        using (ISession session = SessionFactoryX.OpenSession())
        {
            return session.Query<Employee<T>>().ToList();
        }
    }

    public Employee<T> GetById(T id)
    {
        using (ISession session = SessionFactoryX.OpenSession())
        {
            return session.Get<Employee<T>>(id);
        }
    }

}

public class EmployeeBuilder<T>
{
    private T _id;
    private string _firstName;
    private string _lastName;
    private string _position;
    private string _department;
    private string _phoneNumber;

    public void SetId(T id)
    {
        _id = id;
    }

    public void SetFirstName(string firstName)
    {
        _firstName = firstName;
    }

    public void SetLastName(string lastName)
    {
        _lastName = lastName;
    }

    public void SetPosition(string position)
    {
        _position = position;
    }

    public void SetDepartment(string department)
    {
        _department = department;
    }

    public void SetPhoneNumber(string phoneNumber)
    {
        _phoneNumber = phoneNumber;
    }

    public Employee<T> Build()
    {
        return new Employee<T>(_id, _firstName, _lastName, _position, _department, _phoneNumber);
    }
}

public class Program
{
    public static void Main(string[] args)
    {
        using (var session = SessionFactoryX.OpenSession())
        {
            using (var transaction = session.BeginTransaction())
            {
                var empBuilder = new EmployeeBuilder<int>();
                empBuilder.SetId(1);
                empBuilder.SetFirstName("Bartosz");
                empBuilder.SetLastName("Dolas");
                empBuilder.SetPosition("Kierownik");
                empBuilder.SetDepartment("Sprzedaży");
                empBuilder.SetPhoneNumber("456-218-325");
                var employee = empBuilder.Build();

                session.Save(employee);
                transaction.Commit();
            }

            using (var transaction = session.BeginTransaction())
            {
                var employees = session.CreateCriteria<Employee<int>>().List<Employee<int>>();
                foreach (var emp in employees)
                {
                    Console.WriteLine(emp.Format());
                }
                transaction.Commit();
            }
        }

    }
}

Error: (6,7): error CS0246: Nie można znaleźć nazwy typu lub przestrzeni nazw „NHibernate” (brak dyrektywy using lub odwołania do zestawu?)
(7,7): error CS0246: Nie można znaleźć nazwy typu lub przestrzeni nazw „FluentNHibernate” (brak dyrektywy using lub odwołania do zestawu?)
(8,7): error CS0246: Nie można znaleźć nazwy typu lub przestrzeni nazw „NPOI” (brak dyrektywy using lub odwołania do zestawu?)
(10,28): error CS0246: Nie można znaleźć nazwy typu lub przestrzeni nazw „ClassMap<>” (brak dyrektywy using lub odwołania do zestawu?)
(10,46): error CS0246: Nie można znaleźć nazwy typu lub przestrzeni nazw „T” (brak dyrektywy using lub odwołania do zestawu?)
(26,20): error CS0246: Nie można znaleźć nazwy typu lub przestrzeni nazw „ISessionFactory” (brak dyrektywy using lub odwołania do zestawu?)
(28,20): error CS0246: Nie można znaleźć nazwy typu lub przestrzeni nazw „ISessionFactory” (brak dyrektywy using lub odwołania do zestawu?)
(43,19): error CS0246: Nie można znaleźć nazwy

In [6]:
// Zadanie 3 Wykonali: Bartosz, Patryk S

using FluentNHibernate.Cfg;
using FluentNHibernate.Cfg.Db;
using NHibernate;
using NHibernate.Tool.hbm2ddl;

public class Program
{
    private readonly ISessionFactory _mainDbSessionFactory;
    private readonly ISessionFactory _localDbSessionFactory;
    private bool _mainDbAvailable = true;

    public Program(string mainDbConnectionString, string localDbConnectionString) {
        _mainDbSessionFactory = CreateSessionFactory(mainDbConnectionString);
        _localDbSessionFactory = CreateSessionFactory(localDbConnectionString);

    }

    public void LogError(Error error) {
        try {
            if (_mainDbAvailable) {
                using (var session = _mainDbSessionFactory.OpenSession())
                using (var transaction = session.BeginTransaction()) {
                    session.Save(error);
                    transaction.Commit();
                }
            }
            else {
                using (var session = _localDbSessionFactory.OpenSession())
                using (var transaction = session.BeginTransaction()) {
                    session.Save(error);
                    transaction.Commit();
                }
            }
        }
        catch (Exception ex) {
            Console.WriteLine("Błąd podczas zapisu: " + ex.Message);
            _mainDbAvailable = false;
        }
    }

    public void TransferData()
    {
        try {
            using (var mainDbSession = _mainDbSessionFactory.OpenSession())
            using (var localDbSession = _localDbSessionFactory.OpenSession()) {
                var errors = localDbSession.Query<Error>().ToList();
                using (var mainDbTransaction = mainDbSession.BeginTransaction()) {
                    foreach (var error in errors) {
                        mainDbSession.Save(error);
                    }
                    mainDbTransaction.Commit();
                }
                using (var localDbTransaction = localDbSession.BeginTransaction()) {
                    localDbSession.Delete("from Error");
                    localDbTransaction.Commit();
                }
            }
            _mainDbAvailable = true;
        }
        catch (Exception ex) {
            Console.WriteLine("Błąd podczas przesyłu danych: " + ex.Message);
        }
    }

    private ISessionFactory CreateSessionFactory(string connectionString) {
        return Fluently.Configure()
            .Database(SQLiteConfiguration.Standard.ConnectionString(connectionString).ShowSql())
            .Mappings(m => m.FluentMappings.AddFromAssemblyOf<Error>())
            .ExposeConfiguration(cfg => new SchemaExport(cfg).Create(false, true))
            .BuildSessionFactory();
    }
}

public class Error {
    public virtual int Id { get; set; }
    public virtual string Message { get; set; }
    public virtual DateTime Timestamp { get; set; }
    public string StackTrace { get; set; }
}
  